# Finite Markov Decision Processes

### Intro.

MDP는 순차적 의사결정(sequential decision making) 모델로써, 행동(action)은 즉각적인 보상(rewards) 뿐만 아니라 이후 순차적인 상태(states)들의 보상(future rewards)을 고려하여 결정된다. 그러므로 MDP에서는 지연보상(delayed rewards)과 즉각적 보상(immediate rewards) 사이에서의 적절한 균형을 염두할 필요가 있다. 또한 state에 의존적인 문제에 대해 각 action에 대한 장기적인 가치를 정확히 매기기 위해, Multi-armed bandit 문제에서 $q_*(a)$를 추정했던 것에 달리, MDP에서는 state s에 대한 action a의 value인 $q_*(s, a)$와 각 state에 대한 value인 $v_*(s)$를 추정한다.

MDP는 수학적으로 정확히 강화학습 문제를 서술하기 위한 이상적인 형태이다. 이 챕터에서는 문제들을 수학적인 구조로 풀어내기 위해 필요한 주요 요소들인 returns, value functions, Bellman equations 같은 개념들을 소개한다. 더불어 finite MDP로 정형화할 수 있는 몇 가지 응용예시들을 살펴보도록 한다.

### 3.1 The Agent-Environment Interface

<img src="images/fig3.1.png" alt="The agent-environment interaction in MDP"/>

앞서(챕터1) 강화학습을 목표지향적 상호작용 학습(goal-directed learning from interation)이라 표현하였다. MDP를 이용하면 이러한 문제를 단순한 형태로 표현할 수 있게 된다. 여기서 '상호작용'에 대해 좀 더 자세히 살펴보도록 하자. 위 그림 3.1처럼 상호작용은 agent와 environment(환경)사이에서 지속적으로 이루어진다. 이때 *agent*란 의사결정과 학습을 하는 주체를 뜻하며, *environment*란 agent가 상호작용하는 agent외의 모든 것을 의미한다. 

Agent와 environment의 상호작용(interaction)은 순차적(sequential)이고 이산적인(discrete) time step, $t=1,2,3,\dots$ 에 걸쳐서 발생한다. 특정 time step $t$에서 agent는 environment에 대한 상태, $S_t \in \mathcal{S}$ 를 받아 이를 기반으로 action, $A_t \in \mathcal{A}$ 을 선택하게 된다. 그리고 한 time step 뒤, action에 대한 결과로써 agent는 수치적인(numerical) 보상, $R_{t+1} \in \mathcal{R} \subset \mathbb{R}$ 과 다음 상태 $S_{t+1}$를 받는다. 이는 곧 아래와 같은 trajectory로 표현할 수 있다.
$$S_0, A_0, R_1, S_1, A_1, R_2, S_2, A_2, R_3, \dots$$

*Finite* MDP에서 states, actions, rewards ($\mathcal{S}, \mathcal{A}, \mathcal{R}$)에 대한 집합은 유한한 갯수의 원소로 구성되어 있다. 이 경우, 어떤 시점 $t$ 에서 앞선 state $s \in \mathcal{S}$ 와 action $a \in \mathcal{A}$ 에 대해 random variables $S_t$와 $R_t$이 어떤 특정한 값 $s' \in \mathcal{S}, r \in \mathcal{R}$ 을 가질 확률은 다음과 같이 표현된다.
$$p(s', r \:| \: s, a) \doteq Pr\{S_t = s', R_{t} = r \: | \: S_{t-1} = s, A_{t-1} = a \},\\
\text{for all $s', s \in \mathcal{S},$ and $a \in \mathcal{A}(s)$.}
$$

함수 $p: \mathcal{S} \: \times \: \mathcal{R} \: \times \: \mathcal{S} \: \times \: \mathcal{A} \rightarrow [0, 1]$ 는 4개 인자에 대한 ordinary deterministic function이며, 다음을 만족하는 확률분포를 명시한다.
$$
\sum_{s' \in \mathcal{S}} \sum_{r \in \mathcal{R}} p(s', r \:| \: s, a) = 1, \text{ for all } s \in \mathcal{S}, a \in \mathcal{A}(s).
$$

책에서는 이 4개 인자에 대한 $p$ 함수가 주로 사용된다. 더불어 $p$ 함수를 이용해 유도할 수 있는 다른 몇 가지 notation들에 대해서도 추가적으로 살펴보도록 하자.

* *state-transition probabilities: *
$$
p(s' \: | \: s, a) \doteq Pr\{S_t \: | \: S_{t-1}=s, A_{t-1} = a \} = \sum_{r \in \mathcal{R}} p (s', r \: | \: s, a).
$$

* *expected rewards: *
$$
r(s,a) \doteq \mathbb{E}[ R_t \: | \: S_{t-1} = s, A_{t-1} = a ] = \sum_{r \in \mathcal{R}} r \sum_{s' \in \mathcal{S}} p (s', r \: | \: s, a).
$$

* *expected rewards for state-action-next-state triples: *
$$
r(s,a,s') \doteq \mathbb{E}[ R_t \: | \: S_{t-1} = s, A_{t-1} = a, S_t = s' ] = \sum_{r \in \mathcal{R}} r \frac{p (s', r \: | \: s, a)}{p(s' \: | \: s, a)}.
$$

MDP framework는 목표지향적인 상호작용 학습(goal-directed learning from interation) 문제를 agent와 environment 사이에 3가지의 신호가 오가는 형태로 추상화한다. 바로 agent의 의사결정을 나타내는 신호(action), 그 의사결정의 근거가 되는 신호(state), 그리고 agent의 목표를 정의하는 신호(reward)다. 이 framework가 모든 decision-learning 문제를 충분히 잘 표현할 수 있다고 할 수는 없지만, 이 방식의 유용함과 다양한 적용가능성은 이미 널리 입증되어 왔다.

**Example 3.3** Recycling Robot

빈 깡통을 수거하는 로봇이 있다고 해보자. 로봇은 충전식 배터리로 구동되며, 깡통을 감지하면 팔을 이용하여 그것을 집은 뒤 쓰레기통에 집어넣는다. 이때 가장 고차원의 의사결정은 배터리의 잔량에 근거한 강화학습을 통해 이루어진다. 간단한 예시를 들어보자. 배터리는 다음의 두 가지 상태로 구분된다: $\mathcal{S} = \{high, low \}$. 각 상태에서 로봇은 깡통을 탐색(search)하거나, 누군가가 깡통을 집어주길 기다리거나(wait), 충전소로 돌아갈 수 있다(recharge). 상태에 따른 action set은 다음과 같다: $\mathcal{A}(high) = \{search, wait\}$ and $\mathcal{A}(low) = \{search, wait, recharge\}$. 보상은 대부분의 경우 0이지만 깡통을 수거하면 양수($r_\text{search}$)가, 배터리가 바닥나면 음수(-3)가 된다. ($r_\text{search} > r_\text{wait}$가 되도록 설정)

다음은 각 상태간에 천이(transition)가 일어날 확률에 대해 이야기해보자. $high$ 상태에서 탐색을 할때, 배터리 잔량이 여전히 $high$로 유지될 가능성을 $\alpha$, 그렇지 않을 가능성을 $1 - \alpha$라 하겠다. 또한 상태가 $low$일때 탐색 후에도 여전히 배터리 잔량이 $low$로 유지될 가능성은 $\beta$, 배터리가 소진되어 사용자의 손에 의해 충전될(rescued) 가능성은 $1 - \beta$라고 가정한다.

마지막으로 충전소로 돌아가는 길에는 깡통을 수거할 수 없다고 해보자. 앞서 서술한 내용을 finite MDP로 표현해보면 다음과 같다.
<img src="images/example3.3.png" alt="Recycling Robot"/>
Finite MDP는 좌측과 같이 테이블로 표현되거나 혹은 우측처럼 *state node*와 *action node*로 구성된 *transition graph*로도 표현될 수 있다. 여기서 같은 state-action에 대한 transition들의 모든 확률의 총합은 항상 1임을 유의하자.